# Basic Alhazen Agent / Chatbot.

> Building a simple Gradio application to allow the user to chat to Alhazen. 

In [ ]:
#| default_exp apps.chat

In [7]:
#| export

from alhazen.core import MODEL_TYPE, PromptTemplateRegistry, load_alhazen_tool_environment, get_langchain_chatmodel
from alhazen.schema_sqla import *
from alhazen.tools.basic import IntrospectionTool
from alhazen.tools.paperqa_emulation_tool import PaperQAEmulationTool
from alhazen.tools.metadata_extraction_tool import MetadataExtractionTool, MetadataExtractionWithRAGTool 
from alhazen.toolkit import AlhazenToolkit
from alhazen.utils.jats_text_extractor import NxmlDoc
from alhazen.utils.ceifns_db import Ceifns_LiteratureDb, create_ceifns_database, drop_ceifns_database

import click

from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent, AgentOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActJsonSingleInputOutputParser
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain.output_parsers.json import parse_json_markdown
from langchain.pydantic_v1 import BaseModel
from langchain.tools.render import render_text_description, render_text_description_and_args
from langchain.agents.output_parsers.json import JSONAgentOutputParser
from langchain_core.exceptions import OutputParserException
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.language_models.base import BaseLanguageModel
from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain_core.tools import BaseTool


from langchain.prompts import ChatPromptTemplate


import gradio as gr
import os
import pandas as pd
from pathlib import Path
import re
import requests
import sys
from typing import Dict, List, Optional, Sequence, Union, Any
from uuid import UUID

In [2]:
#| export 

class AlhazenAgentChatBot:
    '''Provide access to an Alhazen agent via a simple chat interface with Gradio.'''

    def __init__(self):

        loc, db_name, model_type, model_name = load_alhazen_tool_environment()

        self.db = Ceifns_LiteratureDb(loc=loc, name=db_name)
        #self.llm  = get_langchain_chatmodel(MODEL_TYPE.OpenAI, 'gpt-4-0125-preview')
        self.llm  = get_langchain_chatmodel(model_type, model_name)

        self.tk = AlhazenToolkit(db=self.db, llm=self.llm)
        self.tools = self.tk.get_tools()

        pts = PromptTemplateRegistry()
        pts.load_prompts_from_yaml('alhazen_base.yaml')

        self.prompt = pts.get_prompt_template('structured chat agent').generate_chat_prompt_template()
        
        agent = create_structured_chat_agent_withFixes(self.llm, self.tools, self.prompt)
        #introspection_tool = [t for t in self.tools if isinstance(t, IntrospectionTool)][0]
        #introspection_tool._set_agent(agent)

        self.agent_executor = AgentExecutor(agent=agent, 
                                            tools=self.tools,
                                            verbose=True, 
                                            return_intermediate_steps=True)

    def get_introspection_tool(self):
        return [t for t in self.tools if isinstance(t, IntrospectionTool)][0]

    # We need to add these input/output schemas because the current AgentExecutor
    # is lacking in schemas.
    class ChatRequest(BaseModel):
        question: str
        chat_history: Optional[List[Dict[str, str]]]

    class Input(BaseModel):
        input: str

    class Output(BaseModel):
        output: str

    def run_gradio(self):

        def add_text(history, text):
            #print('add_text: history: %s, text: %s'%(history, text))
            history = history + [(text, None)]
            return history, gr.Textbox(value="", interactive=False)
            
        def clear_chat(history):
            return []

        def bot(history):
            #print('bot: history: %s'%(history))
            # prompt to send to the agent is the last message from the user
            prompt = history[-1][0]
            response = self.agent_executor.invoke({"input": prompt}, config={'callbacks': [ConsoleCallbackHandler()]})
            print('RESPONSE: %s'%(str(response)))
            history[-1][1] = str(response.get('output'))
            print('WHOLE HISTORY: %s'%(history))
            return history

        with gr.Blocks() as demo:
            with gr.Tab("Chat"):
                chatbot = gr.Chatbot(
                    [],
                    elem_id="chatbot",
                    bubble_full_width=False,
                    height="100%",
                    #avatar_images=(None, files(alhazen_resources).joinpath('alhazen.png'))
                )
                with gr.Row():
                    txt = gr.Textbox(
                        scale=4,
                        show_label=False,
                        placeholder="Enter text and press enter.",
                        container=False,
                    )
                    clear_btn = gr.Button("Clear")
            with gr.Tab("Dashboard"):
                with gr.Row():
                    doc_text = gr.HTML(label="File Contents")
            
            txt_msg = txt.submit(add_text, [chatbot, txt], [chatbot, txt], queue=False).then(bot, chatbot, chatbot)
            txt_msg.then(lambda: gr.Textbox(interactive=True), None, [txt], queue=False)                
            clear_btn.click(clear_chat, [], [chatbot], queue=False)
                
        demo.queue()
        demo.launch()

def create_structured_chat_agent_withFixes(
    llm: BaseLanguageModel, tools: Sequence[BaseTool], prompt: ChatPromptTemplate
) -> Runnable:
    """modified from `create_structured_chat_agent` to deal with escaped underscore characters."""

    missing_vars = {"tools", "tool_names", "agent_scratchpad"}.difference(
        prompt.input_variables
    )
    if missing_vars:
        raise ValueError(f"Prompt missing required variables: {missing_vars}")

    prompt = prompt.partial(
        tools=render_text_description_and_args(list(tools)),
        tool_names=", ".join([t.name for t in tools]),
    )
    llm_with_stop = llm.bind(stop=["Observation"])

    agent = (
        RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]),
        )
        | prompt
        | llm_with_stop
        | JSONAgentOutputParser_withFixes()
    )
    return agent

class JSONAgentOutputParser_withFixes(AgentOutputParser):
    """modified from `JSONAgentOutputParser` to deal with escaped underscore characters."""

    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        try:
            # Hack to remove escaped underscores.
            text = re.sub('\\\\_', '_', text)
            text = re.sub('\\\\_', '_', text)
            response = parse_json_markdown(text)
            if isinstance(response, list):
                # gpt turbo frequently ignores the directive to emit a single action
                response = response[0]
            if response["action"] == "Final Answer":
                return AgentFinish({"output": response["action_input"]}, text)
            else:
                return AgentAction(
                    response["action"], response.get("action_input", {}), text
                )
        except Exception as e:
            raise OutputParserException(f"Could not parse LLM output: {text}") from e

    @property
    def _type(self) -> str:
        return "json-agent"
    


In [1]:
#| export
@click.command()
@click.option('--loc', default=None, help='Where on disk to put local files')
@click.option('--db_name', prompt=None, help='The name of the database being used.')
def alhazen_chatbot(loc, db_name):
    """Runs the Alhazen Chatbot."""
    if loc:
        os.environ['LOCAL_FILE_PATH'] = loc
    if db_name:
        os.environ['ALHAZEN_DB_NAME'] = db_name
    cb = AlhazenAgentChatBot()
    cb.run_gradio()


NameError: name 'click' is not defined

In [8]:
#| export
#|eval: false

if __name__ == '__main__':
    alhazen_chatbot()

Usage: ipykernel_launcher.py [OPTIONS]
Try 'ipykernel_launcher.py --help' for help.

Error: No such option: --f


AttributeError: 'tuple' object has no attribute 'tb_frame'